In [1]:
import pandas as pd
import numpy as np
import math as maths
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from matplotlib.patches import FancyArrow
import scipy.interpolate
import netCDF4 as nc
import xarray as xr
import scipy.ndimage
import scipy as sp
import alphashape
from numpy.linalg import eig
from matplotlib import ticker, cm
from scipy import interpolate
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from matplotlib import style
from pathlib import Path  
style.use('default') or plt.style.use('default')
import seaborn as sns


In [2]:
# Read in surface strain rate data

# E11, E22, E33, E12, E23, E31, Eii (trace)

SurfStrainRate = pd.read_csv('../Simulations/CurrentTests/BasalFriction2e7StrainRate/StrainRate.csv', sep=",")


FileNotFoundError: [Errno 2] No such file or directory: '../Simulations/CurrentTests/BasalFriction2e7StrainRate/StrainRate.csv'

In [ ]:
# Create a regular grid and interpolate the data onto it

xmin, xmax = min(SurfStrainRate['Points:0']), max(SurfStrainRate['Points:0'])
ymin, ymax = min(SurfStrainRate['Points:1']), max(SurfStrainRate['Points:1'])
dist = 100
nx, ny = int((xmax - xmin)/dist + 1), int((ymax - ymin)/dist + 1)
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)
grid_x, grid_y = np.meshgrid(x, y)

E11 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 1"], (grid_x, grid_y), method='linear')
E22 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 2"], (grid_x, grid_y), method='linear')
E33 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 3"], (grid_x, grid_y), method='linear')
E12 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 4"], (grid_x, grid_y), method='linear')
E23 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 5"], (grid_x, grid_y), method='linear')
E31 = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 6"], (grid_x, grid_y), method='linear')
Eii = scipy.interpolate.griddata((SurfStrainRate["Points:0"], SurfStrainRate["Points:1"]), SurfStrainRate["strainrate 7"], (grid_x, grid_y), method='linear')


In [ ]:
# Calculate the eigenvalues and eigenvectors of the strain rate for the surface.
# Later also do this for the isochrones

def Eigs(exx, eyy, exy):
    v11 = np.zeros_like(exx) # First component of the first vector
    v12 = np.zeros_like(exx) # Second component of the first vector
    v21 = np.zeros_like(exx) # First component of the second vector
    v22 = np.zeros_like(exx) # Second component of the second vector
    e1 = np.zeros_like(exx)
    e2 = np.zeros_like(exx)
    I = len(exx)
    J = len(exx[0])
    for i in range(1, I-1):
        for j in range(1, J-1):
            if ((maths.isnan(exx[i][j])) or (maths.isnan(eyy[i][j])) or (maths.isnan(exy[i][j]))):
                e1[i][j], e2[i][j] = np.nan, np.nan
                v11[i][j], v12[i][j] = np.nan, np.nan
                v21[i][j], v22[i][j] = np.nan, np.nan
            else:
                arr = np.array([[exx[i][j], exy[i][j]], 
                                [exy[i][j], eyy[i][j]]])
                w,v = eig(arr)
                e1[i][j], e2[i][j] = w
                v11[i][j], v12[i][j] = v[0]
                v21[i][j], v22[i][j] = v[1]
    return e1, e2, v11, v12, v21, v22

e1_1, e2_1, v11_1, v12_1, v21_1, v22_1 = Eigs(E11, E22, E12)



In [ ]:
# Make an array of the eigenvector/value corresponding with the largest eigenvalue

def MinMaxEigs(e1, e2, v11, v12, v21, v22):
    e_min = np.zeros_like(e1)
    e_max = np.zeros_like(e1)
    v1_min = np.zeros_like(e1)
    v2_min = np.zeros_like(e1)
    v1_max = np.zeros_like(e1)
    v2_max = np.zeros_like(e1)
    I = len(e1)
    J = len(e1[0])
    for i in range(1, I-1):
        for j in range(1, J-1):
            if e1[i][j] > e2[i][j]:
                e_min[i][j] = e2[i][j]
                e_max[i][j] = e1[i][j]
                v1_min[i][j] = v21[i][j]
                v2_min[i][j] = v22[i][j]
                v1_max[i][j] = v11[i][j]
                v2_max[i][j] = v12[i][j]
            else:
                e_min[i][j] = e1[i][j]
                e_max[i][j] = e2[i][j]
                v1_min[i][j] = v11[i][j]
                v2_min[i][j] = v12[i][j]
                v1_max[i][j] = v21[i][j]
                v2_max[i][j] = v22[i][j]
#    for i in range(1, I-1):
#        for j in range(1, J-1):
#            if v2_min[i][j] < 0:
#                v1_min[i][j] = - v1_min[i][j]
#                v2_min[i][j] = - v2_min[i][j]
#                v1_max[i][j] = - v1_max[i][j]
#                v2_max[i][j] = - v2_max[i][j]
    return e_min, e_max, v1_min, v2_min, v1_max, v2_max

e_min_1, e_max_1, v1_min_1, v2_min_1, v1_max_1, v2_max_1 = MinMaxEigs(e1_1, e2_1, v11_1, v12_1, v21_1, v22_1)


In [ ]:
e_min_1

In [ ]:
e_max_1

In [ ]:
# Calculate the strain rate magnitude

def StrainMag(exx, eyy):
    e_total = np.zeros_like(exx)
    I = len(exx)
    J = len(exx[0])
    for i in range(1, I-1):
        for j in range(1, J-1):
            e_total[i][j] = (exx[i][j] + eyy[i][j])/2
    return e_total

e_total_1 = StrainMag(E11, E22)

In [ ]:
# Make two copies of the strain rate data and 
# Load in grounding line and make two copies of the data: inside and outside grounding line

dfGL = pd.read_csv("../Data/GroundingLine.csv")

points = np.column_stack((list(dfGL["Points:0"])[::10], list(dfGL["Points:1"])[::10]))

# [list(dfGL["Points:0"]), list(dfGL["Points:1"])]
alpha = 0.95 * alphashape.optimizealpha(points)
hull = alphashape.alphashape(points, alpha)
hull_pts = hull.exterior.coords.xy

poly = Polygon(zip(list(hull_pts[0]), list(hull_pts[1])))

e_max_1_in = e_max_1.copy()
e_max_1_out = e_max_1.copy()


poly

In [ ]:
for i in range(len(e_max_1)):
    for j in range(len(e_max_1[0])):
        boolean = poly.contains(Point(grid_x[i][j], grid_y[i][j]))
        if boolean == False:
            e_max_1_in[i][j] = np.nan
            E33[i][j] = np.nan
#            v1_max_1 = np.nan
#            v2_max_1 = np.nan
        if boolean == True:
            e_max_1_out[i][j] = np.nan
            

In [ ]:
# Plot the magnitude of the larger eigenvalue

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
#levels1 = np.linspace(0.01, 0.002, 500)
levels2 = np.linspace(-0.003, 0.0, 500)
CS = ax.contourf(grid_x, grid_y, E33, levels=levels2, cmap=plt.cm.Spectral, style="sci", extend='both')
#CS1 = ax.contourf(grid_x, grid_y, e_max_1_in, levels=levels1, cmap=plt.cm.GnBu, extend='max', style="sci")
#CS2 = ax.contourf(grid_x, grid_y, e_max_1_out, levels=levels2, cmap=plt.cm.YlOrRd, extend='max', style="sci")
skip = (slice(None, None, 15), slice(None, None, 15))
#strm = ax.streamplot(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], density=[5, 5],\
#                     linewidth=0.5, arrowsize=0, color='black')
ax.quiver(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], scale=50, pivot='mid', headlength=0, headwidth=1)
#ax.plot(dfGL['Points:0'], dfGL['Points:1'], color='black')
ax.set_xlabel(r'$x$ [km]', size=20)
ax.set_ylabel(r'$y$ [km]', size=20)
ax.set_xlim([940000, 985000])
ax.set_ylim([1910000, 1960000])
cbar = fig.colorbar(CS)
cbar.set_label('Principal strain rate [a$^{-1}$]', size=20)
#fig.savefig('DerwaelStrainRate.jpg', format='jpg', dpi=700, bbox_inches = "tight")


In [ ]:
type(grid_x)

In [ ]:
# Plot the magnitude of the larger eigenvalue

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
#levels1 = np.linspace(0.01, 0.002, 500)
levels2 = np.linspace(0.0, 0.01, 500)
CS = ax.contourf(grid_x, grid_y, e_max_1, levels=levels2, locator=ticker.LogLocator(), cmap=plt.cm.GnBu, extend='max', style="sci")
#CS1 = ax.contourf(grid_x, grid_y, e_max_1_in, levels=levels1, cmap=plt.cm.GnBu, extend='max', style="sci")
#CS2 = ax.contourf(grid_x, grid_y, e_max_1_out, levels=levels2, cmap=plt.cm.YlOrRd, extend='max', style="sci")
skip = (slice(None, None, 15), slice(None, None, 15))
#strm = ax.streamplot(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], density=[5, 5],\
#                     linewidth=0.5, arrowsize=0, color='black')
ax.quiver(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], scale=50, pivot='mid', headlength=0, headwidth=1)
#ax.plot(dfGL['Points:0'], dfGL['Points:1'], color='black')
ax.set_xlabel(r'$x$ [km]', size=20)
ax.set_ylabel(r'$y$ [km]', size=20)
ax.set_xlim([940000, 985000])
ax.set_ylim([1910000, 1960000])
cbar = fig.colorbar(CS)
cbar.set_label('Principal strain rate [a$^{-1}$]', size=20)
#fig.savefig('DerwaelStrainRate.jpg', format='jpg', dpi=700, bbox_inches = "tight")


In [ ]:
# Plot the magnitude of the larger eigenvalue

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
#levels1 = np.linspace(0.01, 0.002, 500)
levels2 = np.linspace(0.0, 0.015, 500)
CS = ax.contourf(grid_x, grid_y, e_max_1, levels=levels2, locator=ticker.LogLocator(), cmap=plt.cm.GnBu, extend='max', style="sci")
#CS1 = ax.contourf(grid_x, grid_y, e_max_1_in, levels=levels1, cmap=plt.cm.GnBu, extend='max', style="sci")
#CS2 = ax.contourf(grid_x, grid_y, e_max_1_out, levels=levels2, cmap=plt.cm.YlOrRd, extend='max', style="sci")
skip = (slice(None, None, 15), slice(None, None, 15))
#strm = ax.streamplot(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], density=[5, 5],\
#                     linewidth=0.5, arrowsize=0, color='black')
ax.quiver(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], scale=50, pivot='mid', headlength=0, headwidth=1)
#ax.plot(dfGL['Points:0'], dfGL['Points:1'], color='black')
ax.set_xlabel(r'$x$ [km]', size=20)
ax.set_ylabel(r'$y$ [km]', size=20)
ax.set_xlim([940000, 985000])
ax.set_ylim([1910000, 1960000])
cbar = fig.colorbar(CS)
cbar.set_label('Principal strain rate [a$^{-1}$]', size=20)
#fig.savefig('DerwaelStrainRate.jpg', format='jpg', dpi=700, bbox_inches = "tight")


In [ ]:
# Plot the magnitude of the larger eigenvalue

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
#levels1 = np.linspace(0.01, 0.002, 500)
levels2 = np.linspace(0.0, 0.02, 500)
CS = ax.contourf(grid_x, grid_y, e_max_1, levels=levels2, locator=ticker.LogLocator(), cmap=plt.cm.GnBu, extend='max', style="sci")
#CS1 = ax.contourf(grid_x, grid_y, e_max_1_in, levels=levels1, cmap=plt.cm.GnBu, extend='max', style="sci")
#CS2 = ax.contourf(grid_x, grid_y, e_max_1_out, levels=levels2, cmap=plt.cm.YlOrRd, extend='max', style="sci")
skip = (slice(None, None, 15), slice(None, None, 15))
#strm = ax.streamplot(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], density=[5, 5],\
#                     linewidth=0.5, arrowsize=0, color='black')
ax.quiver(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], scale=50, pivot='mid', headlength=0, headwidth=1)
#ax.plot(dfGL['Points:0'], dfGL['Points:1'], color='black')
ax.set_xlabel(r'$x$ [km]', size=20)
ax.set_ylabel(r'$y$ [km]', size=20)
ax.set_xlim([940000, 985000])
ax.set_ylim([1910000, 1960000])
cbar = fig.colorbar(CS)
cbar.set_label('Principal strain rate [a$^{-1}$]', size=20)
fig.savefig('DerwaelStrainRate.jpg', format='jpg', dpi=700, bbox_inches = "tight")


In [ ]:
# Plot the magnitude of the larger eigenvalue

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
#levels1 = np.linspace(0.01, 0.002, 500)
levels2 = np.linspace(0.0001, 0.02, 500)
CS = ax.contourf(grid_x, grid_y, e_max_1, levels=levels2, locator=ticker.LogLocator(), cmap=plt.cm.plasma_r, extend='both', style="sci")

#plasma_r

#CS1 = ax.contourf(grid_x, grid_y, e_max_1_in, levels=levels1, cmap=plt.cm.GnBu, extend='max', style="sci")
#CS2 = ax.contourf(grid_x, grid_y, e_max_1_out, levels=levels2, cmap=plt.cm.YlOrRd, extend='max', style="sci")
skip = (slice(None, None, 15), slice(None, None, 15))
#strm = ax.streamplot(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], density=[5, 5],\
#                     linewidth=0.5, arrowsize=0, color='black')
ax.quiver(grid_x[skip], grid_y[skip], v1_max_1[skip], v2_max_1[skip], scale=50, pivot='mid', headlength=0, headwidth=1)
#ax.plot(dfGL['Points:0'], dfGL['Points:1'], color='black')
ax.set_xlabel(r'$x$ [km]', size=20)
ax.set_ylabel(r'$y$ [km]', size=20)
ax.set_xlim([940000, 985000])
ax.set_ylim([1910000, 1960000])
cbar = fig.colorbar(CS)
cbar.set_label('Principal strain rate [a$^{-1}$]', size=20)
#fig.savefig('DerwaelStrainRate.jpg', format='jpg', dpi=700, bbox_inches = "tight")


In [6]:
a2D = np.array([[1, 0], [0, 1]])

scipy.ndimage.interpolation.rotate(a2D, 90, axes=(1, 0))


/var/folders/7m/lgqxn0610bs8sxwnf4k6z_hw0000gn/T/ipykernel_49131/2177416956.py:3: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  scipy.ndimage.interpolation.rotate(a2D, 90, axes=(1, 0))


array([[0, 1],
       [1, 0]])